In [36]:
import sys
import os
import dotenv

dotenv.load_dotenv(dotenv.find_dotenv())

sys.path.insert(0, os.path.abspath('..'))

from hyperdemocracy import load_assembleco_records

df = load_assembleco_records()

Found cached dataset parquet (/Users/forrestmurray/.cache/huggingface/datasets/assembleco___parquet/assembleco--hyperdemocracy-a598a9b2b17e51dc/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


In [37]:
import tiktoken

tokenizer = tiktoken.get_encoding('cl100k_base')

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

In [38]:
from hyperdemocracy import get_aco_split_docs, get_legislative_documents_from_df

df = get_legislative_documents_from_df(df)

split_docs = get_aco_split_docs(df)

In [39]:
from langchain.embeddings.openai import OpenAIEmbeddings
from hyperdemocracy import get_config

config = get_config()
embeddings = OpenAIEmbeddings(model=config['embd'], disallowed_special=())

In [40]:
import pinecone 
from langchain.vectorstores import Pinecone

pinecone.init(
    api_key=os.environ['PINECONE_API_KEY'],
    environment=os.environ['PINECONE_ENV']
)

index_name = "hyperdemocracy"


# Saving Embeddings in Pinecone

This will take a while!

In [41]:
from tqdm import tqdm 
import time

batch_size = 128
for ii in tqdm(range(0, len(split_docs), batch_size)):
    batch = split_docs[ii:ii+batch_size]
    if ii == 0:
        vec_store = Pinecone.from_documents(split_docs, embeddings, index_name=index_name)
    else:
        vec_store.add_documents(batch)
    time.sleep(1.0)
# vec_store.persist()

  0%|          | 0/1435 [00:00<?, ?it/s]